In [1]:
import pandas as pd

In [2]:
from pandas import ExcelWriter
from pandas import ExcelFile

In [3]:
import re

In [4]:
df_Quotes = pd.read_excel('C:\\Users\\slice\\NLP POC\\NLP_Quotation\\Input\\Quotes.xlsx')

In [5]:
df_Quotes

,Item No,Description,Count,Quote Sent
0,1,Graphite Stick Pencils – Black – b2,1000,1100
1,2,Graphite Stick Pencils – Red – b2,1500,1100
2,3,Charcoal Pencils – Black,2500,1100
3,4,Color Pencils – Yellow,3000,1100
4,5,Graphite Stick Pencils – Black – b2,1000,1101
5,6,Graphite Stick Pencils – Red – b2,1500,1101


In [6]:
df_Supplier = pd.read_excel('C:\\Users\\slice\\NLP POC\\NLP_Quotation\\Input\\Supplier.xlsx')

In [7]:
df_Supplier

,Supplier No,Supplier Name,Supplier Email
0,1,Abhijith,abhijith.codepixer@outlook.com
1,2,Shailendra,s.shailendravarma@gmail.com


In [77]:
writer = pd.ExcelWriter('C:\\Users\\slice\\NLP POC\\NLP_Quotation\\Input\\Received Quotes.xlsx',engine='openpyxl')

In [4]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [5]:
# CHANGING THE RULES
from spacy.pipeline import SentenceSegmenter

def split_on_newlines(doc):
    start = 0 
    seen_newline = False
    for word in doc:
        if seen_newline:
            yield doc[start:word.i]
            start = word.i
            seen_newline = False
        elif word.text.startswith('\n'): # handles multiple occurrences
            seen_newline = True
    yield doc[start:]      # handles the last group of tokens


sbd = SentenceSegmenter(nlp.vocab, strategy=split_on_newlines)
nlp.add_pipe(sbd)

In [143]:
class CurrencyPairPipeline(object):
    def __init__(self, nlp):
        self.label_hash = nlp.vocab.strings["CURRENCY_PAIR"]
 
    def __call__(self, doc):
        import re
        from spacy.tokens import Span
        new_tokens = doc.ents
        for idx, token in enumerate(doc):
            #Checking if a currency pair, e.g. USDEUR, EURUSD, etc    
            if re.search(r"((USD|EUR|GBP|JPY|CAD)+[A-Z]{3})|([A-Z]{3}(USD|EUR|GBP|JPY|CAD))", token.text, re.IGNORECASE):
                #We found a match so need to update the entities
                span = Span(doc, idx, idx+1, label=self.label_hash)
                #Spacy only supports one label per phrase, so need to conditionally replace (e.g. USDEUR may be wrongly labelled as an ORG or GPE)
                new_tokens = tuple([t for t in new_tokens if t.start != idx]) + (span,)
        doc.ents = new_tokens
        return doc

In [144]:
ccy_pipeline = CurrencyPairPipeline(nlp)
nlp.add_pipe(ccy_pipeline, name='ccy_pipeline', last=True)

In [30]:
#myfile = open('C:\\Users\\slice\\OneDrive\\Desktop\\Template3.txt')
import win32com.client

outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

#inbox = outlook.GetDefaultFolder(6) # "6" refers to the index of a folder - in this case,
                                    # the inbox. You can change that number to reference
                                    # any other folder

for folder in outlook.Folders:
    print("****Folder Name**********************************")
    print(folder)
    
    print("*************************************************")
    for folder1 in folder.Folders:
        if str(folder1)=='POC':
            req_folder=folder1
messages = req_folder.Items
message = messages.GetFirst()
body_content = message.HTMLBody


****Folder Name**********************************
shailu.codepixer@outlook.com
*************************************************


In [31]:
MyFile_STR=body_content

In [34]:
MyFile_STR=MyFile_STR.replace('\r','')

In [33]:
MyFile_STR=MyFile_STR.replace('\n','')

In [11]:
MyFile_STR=MyFile_STR.rsplit('shailu.codepixer@outlook.com')[0]

In [10]:
from bs4 import BeautifulSoup

In [35]:
MyFile_STR

'<html><head><style type="text/css" style="display:none;"> P {margin-top:0;margin-bottom:0;} </style></head><body dir="ltr"><div style="font-family: Calibri, Arial, Helvetica, sans-serif; font-size: 12pt; color: rgb(0, 0, 0);"><table style="font-family: Calibri, Arial, Helvetica, sans-serif; background-color: rgb(255, 255, 255); border-collapse: collapse"><tbody><tr style="background-color: rgb(255, 255, 255)"><td style="width: 120px; border-width: 1px; border-style: solid; border-color: rgb(171, 171, 171)">Item Name</td><td style="width: 120px; border-width: 1px; border-style: solid; border-color: rgb(171, 171, 171)"><span style="margin: 0px; font-size: 15px; font-family: &quot;Segoe UI&quot;, &quot;Segoe UI Web (West European)&quot;, &quot;Segoe UI&quot;, -apple-system, BlinkMacSystemFont, Roboto, &quot;Helvetica Neue&quot;, sans-serif; color: rgb(33, 33, 33); text-align: left; background-color: rgb(255, 255, 255); display: inline !important">Quantity</span><br></td><td style="width:

In [58]:
if "<table" in MyFile_STR:
item_regex='Graphite Stick Pencils –  Black – b2|'
is_count=False
    soup = BeautifulSoup(MyFile_STR, "html.parser")
    for body in soup("tbody"):
        body.unwrap()

    df = pd.read_html(str(soup), flavor="bs4")

    for df1 in df:
        item_col_no=-1
        cost_col_no=-1
        c=0
        for col in df1.columns:
            if is_cost(col):
                cost_col_no=c
                is_count= is_per(col)
                currency=get_currency(col)
            c=c+1
        
        if cost_col_no!=-1 and currency==0:
            currency=get_currency()
            
        
        
        c=0
        for index, row in df1.iterrows():
            for col in row:
                if col+'|' in item_regex:
                    item_col_no=c
            c=c+1
        
        

0
1
2
Graphite Stick Pencils –  Black – b2
1000
$2000
0
1
2
Graphite Stick Pencils – Red – b2
1500
$3000
0
1
2
Color Pencils
3000
$2000
0
1
Abhijith S M abhijithsm94@outlook.com Phone: + 91 - 9940243959 1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India
nan
Abhijith S M abhijithsm94@outlook.com Phone: + 91 - 9940243959 1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India
nan
Abhijith S M abhijithsm94@outlook.com Phone: + 91 - 9940243959 1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India
nan
nan
nan
Abhijith S M
nan
abhijithsm94@outlook.com Phone: + 91 - 9940243959 1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India
abhijithsm94@outlook.com Phone: + 91 - 9940243959 1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India
nan
nan
nan
nan
nan
nan
0
1
Abhijith S M abhijith

In [17]:
def is_cost(text):
    Flag=0
    df_strings= pd.read_excel('C:\\Users\\slice\\NLP POC\\NLP_Quotation\\Input\\Cost.xlsx')
    for index, row in df_strings.iterrows():
        if str(row['String']).lower() in text.lower():
            Flag=1
            return True;
    if Flag==0:
        return False;

In [28]:
    import requests
    import pandas as pd
    from bs4 import BeautifulSoup
    
    class HTMLTableParser:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(table['id'],self.parse_html_table(table))\
                    for table in soup.find_all('table')]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [214]:
re.search(r'[1-9](\d+)*(,\d{1,})*([\.][\d]+)?',MyFile_STR)

<re.Match object; span=(10, 14), match='1000'>

In [180]:
get_nearest_no(MyFile_STR)

'2000'

In [159]:
def get_nearest_no(text):
    c=0
    for token in nlp(text):
        if get_currency(token.text)!=0:
            currency_token=c
            break;
        c=c+1
    c=0
    Nearest_No=''
    for token in nlp(text):
        if token.tag_=='CD':
            if Nearest_No=='':
                Nearest_No=token.text
                diff=abs(currency_token-c)
            else:
                if diff>abs(currency_token-c):
                    Nearest_No=token.text
                    diff=abs(currency_token-c)
        c=c+1
    return Nearest_No;
    

In [155]:
for sent in nlp(MyFile_STR).sents:
    print(sent.text)
    
    #for ent in nlp(sent.text).ents:
        #print(ent.text, ent.start, ent.end, ent.start_char, ent.end_char, ent.label_)
    
    for token in nlp(sent.text):
        #print(token.tag_)
        print(f'{token.text:{10}} {token.pos_:{8}} {token.tag_:{6}} {spacy.explain(token.tag_)}')
        
    print('next sen')

Graphite Stick Pencils – Red – b2 – 1500 Numbers - 3000$
Graphite   PROPN    NNP    noun, proper singular
Stick      PROPN    NNP    noun, proper singular
Pencils    NOUN     NNS    noun, plural
–          PUNCT    :      punctuation mark, colon or ellipsis
Red        PROPN    NNP    noun, proper singular
–          PUNCT    :      punctuation mark, colon or ellipsis
b2         NOUN     NN     noun, singular or mass
–          PUNCT    :      punctuation mark, colon or ellipsis
1500       NUM      CD     cardinal number
Numbers    PROPN    NNP    noun, proper singular
-          PUNCT    HYPH   punctuation mark, hyphen
3000       NUM      CD     cardinal number
$          SYM      $      symbol, currency
next sen


In [ ]:
.000

In [239]:
MyFile_STR=MyFile_STR.replace('\r','\n')

In [240]:
MyFile_STR

'Cost of 1000 Graphite Stick Pencils – Black – b2 is 20$ per item  \n\nCost of 1000 Graphite Stick Pencils – Red – b2 – is 15$ per item\u200b\n\n\u200b\n\nCost of 2500 Charcoal Pencils – Black – is 12$ per item\n\n\n\n\n\n\n\n\n\n\n\n\n\nRegards,\n\n\n\n\n\n\t\n\nAbhijith S M\n\n\n\n\t\n\n\t\n\nabhijithsm94@outlook.com\n\nPhone: + 91 - 9940243959\n\n1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India\n\n\t\n\n <https://twitter.com/ABHIJITH_SM>    <https://www.linkedin.com/in/abhijith-sm-723392184/>   <https://codetwocdn.azureedge.net/images/mail-signatures/generator/abstract-green/fb.png>  <https://www.youtube.com/user/CodeTwoChannel>  \n\n\t\n\n <https://codetwocdn.azureedge.net/images/mail-signatures/generator/abstract-green/pic-2.png> \t\n\n\n\n\t\n\n________________________________\n\n\n\nFrom: Shailendra Varma Sagi <shailu.codepixer@outlook.com>\n\nSent: Friday, February 14, 2020 11:43 PM\n\nTo: Abhijith.Codepixer@outlook.com <Abhijith.

In [40]:
messages = req_folder.Items
message = messages.GetFirst()
body_content = message.body

In [116]:
body_content

'\r\nHi ABC,\r\n\r\nThank you \r\n\r\nGraphite Stick Pencils – Black – b2 – 1000 Numbers\u200b - 2000 USD\r\n\u200b\r\nGraphite Stick Pencils – Red – b2 – 1500 Numbers\u200b - 3000 USD\r\n\u200b\r\nCharcoal Pencils – Black – 2500\u200b - $1500\r\n\r\n\r\n\r\n\r\n\r\nRegards,\r\n\r\n\r\n\t\r\nAbhijith S M\r\n\r\n\t\r\n\t\r\nabhijithsm94@outlook.com\r\nPhone: + 91 - 9940243959\r\n1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India\r\n\t\r\n <https://twitter.com/ABHIJITH_SM>    <https://www.linkedin.com/in/abhijith-sm-723392184/>   <https://codetwocdn.azureedge.net/images/mail-signatures/generator/abstract-green/fb.png>  <https://www.youtube.com/user/CodeTwoChannel>  \r\n\t\r\n <https://codetwocdn.azureedge.net/images/mail-signatures/generator/abstract-green/pic-2.png> \t\r\n\r\n\t\r\n________________________________\r\n\r\nFrom: Shailendra Varma Sagi <shailu.codepixer@outlook.com>\r\nSent: Friday, February 14, 2020 11:43 PM\r\nTo: Abhijith.Cod

In [108]:
myfile.seek(0)

NameError: name 'myfile' is not defined

In [15]:
print(messages)


<COMObject <unknown>>


In [241]:
len(MyFile_STR.rsplit('Shailendra Varma Sagi <shailu.codepixer@outlook.com>'))

2

In [242]:
MyFile_STR.rsplit('Shailendra Varma Sagi <shailu.codepixer@outlook.com>')[0]

'Cost of 1000 Graphite Stick Pencils – Black – b2 is 20$ per item  \n\nCost of 1000 Graphite Stick Pencils – Red – b2 – is 15$ per item\u200b\n\n\u200b\n\nCost of 2500 Charcoal Pencils – Black – is 12$ per item\n\n\n\n\n\n\n\n\n\n\n\n\n\nRegards,\n\n\n\n\n\n\t\n\nAbhijith S M\n\n\n\n\t\n\n\t\n\nabhijithsm94@outlook.com\n\nPhone: + 91 - 9940243959\n\n1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India\n\n\t\n\n <https://twitter.com/ABHIJITH_SM>    <https://www.linkedin.com/in/abhijith-sm-723392184/>   <https://codetwocdn.azureedge.net/images/mail-signatures/generator/abstract-green/fb.png>  <https://www.youtube.com/user/CodeTwoChannel>  \n\n\t\n\n <https://codetwocdn.azureedge.net/images/mail-signatures/generator/abstract-green/pic-2.png> \t\n\n\n\n\t\n\n________________________________\n\n\n\nFrom: '

In [15]:
 x=re.search(r'\nQuote NO: #[\d]{4}\n',MyFile_STR.rsplit('Shailendra Varma Sagi <shailu.codepixer@outlook.com>')[1]).group()

IndexError: list index out of range

In [220]:
Quote=re.search(r'[\d]{4}',x).group()

In [221]:
Quote

'1100'

In [122]:
 x=re.search(r'\nSupplier Name: [\w{8,16}]*\n',MyFile_STR.rsplit('Shailendra Varma Sagi <shailu.codepixer@outlook.com>')[1]).group()

In [123]:
supplier=x.replace('\nSupplier Name:','').replace('\n','')

In [124]:
supplier

' Abhijith'

In [14]:
df_Quotes = df_Quotes[ df_Quotes['Quote Sent'] == int(Quote) ]


NameError: name 'Quote' is not defined

In [13]:
df_Quotes


,Item No,Description,Count,Quote Sent
0,1,Graphite Stick Pencils – Black – b2,1000,1100
1,2,Graphite Stick Pencils – Red – b2,1500,1100
2,3,Charcoal Pencils – Black,2500,1100
3,4,Color Pencils – Yellow,3000,1100
4,5,Graphite Stick Pencils – Black – b2,1000,1101
5,6,Graphite Stick Pencils – Red – b2,1500,1101


In [194]:
QuoteRegex=''
for index,row in df_Quotes.iterrows():
    if QuoteRegex=='':
        QuoteRegex=row['Description']
    else:
        QuoteRegex=QuoteRegex +'|'+ row['Description']

In [195]:
QuoteRegex

'Graphite Stick Pencils – Black – b2|Graphite Stick Pencils – Red – b2|Charcoal Pencils – Black|Color Pencils – Yellow'

In [222]:
Output_df = pd.DataFrame(columns=['Supplier Name', 'QuoteID', 'Item Name','Quote Sent'])


In [223]:
Output_df

,Supplier Name,QuoteID,Item Name,Quote Sent


In [247]:
doc2=nlp(MyFile_STR.rsplit('Shailendra Varma Sagi <shailu.codepixer@outlook.com>')[0])

In [248]:
for sent in doc2.sents:
    print(sent.text)
    
    #for ent in nlp(sent.text).ents:
        #print(ent.text, ent.start, ent.end, ent.start_char, ent.end_char, ent.label_)
        
    print('next sen')

Cost of 1000 Graphite Stick Pencils – Black – b2 is 20$ per item  

Cost of 1000 Graphite Stick Pencils – Red – b2 – is 15$ per item​


next sen
​


next sen
Cost of 2500 Charcoal Pencils – Black – is 12$ per item














next sen
Regards,





	


next sen
Abhijith S M



	

	


next sen
abhijithsm94@outlook.com


next sen
Phone: + 91 - 9940243959


next sen
1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India

	

 
next sen
<https://twitter.com/ABHIJITH_SM>    <https://www.linkedin.com/in/abhijith-sm-723392184/>   <https://codetwocdn.azureedge.net/images/mail-signatures/generator/abstract-green/fb.png>  <https://www.youtube.com/user/CodeTwoChannel>  

	

 <https://codetwocdn.azureedge.net/images/mail-signatures/generator/abstract-green/pic-2.png> 	



	

________________________________




next sen
From:
next sen


In [249]:

for sent in doc2.sents:
    Received_Quote=False
    print(sent.text)
    if len(re.findall(QuoteRegex,sent.text))==0 or sent.text =='\n' :
        print(sent.text)
        print(" No  Next Sentence Please\n")
    else:
        if len(re.findall(QuoteRegex,sent.text))==1:
            if is_per(sent.text):
                Count=int(df_Quotes[ df_Quotes['Description'] == re.search(QuoteRegex,sent.text).group() ]['Count'].iloc[0])
            else:
                Count=1
            Money_list=[ent for ent in nlp(sent.text).ents if ent.label_ == 'MONEY']
            print(Money_list)
            if len(Money_list)>=1:
                if len(Money_list)==1:
                    for ent in nlp(sent.text).ents:
                        if ent.label_ == 'MONEY':
                            if len(re.findall(r'[1-9](\d+)*(,\d{1,})*([\.][\d]+)?',ent.text))==1:
                                Quote_sent=str(round((float(re.search(r'[1-9](\d+)*(,\d{1,})*([\.][\d]+)?',ent.text).group().replace(',',''))*int(Count)),2))
                                if get_currency(sent.text) != 0:
                                    Quote_sent=get_currency(sent.text)+Quote_sent
                                else:
                                    print('Unidentified currency please add it to the data base')
                                Received_Quote=True
                            else:
                                print('Unable to find actual money in money entity')
                elif len(re.findall(r'[1-9](\d+)*(,\d{1,})*([\.][\d]+)?',sent.text))>=1 and get_currency(sent.text)!=0:
                    Quote_sent=str(round((float(get_nearest_no(sent.text).replace(',',''))*int(Count)),2))
                    Quote_sent=get_currency(sent.text)+Quote_sent
                    Received_Quote=True
                else:
                    print("Unable to get currency from the string please check")
            elif len(re.findall(r'[1-9](\d+)*(,\d{1,})*([\.][\d]+)?',sent.text))==1 and get_currency(sent.text)!=0:
                Quote_sent=str(round(float(re.search(r'[1-9](\d+)*(,\d{1,})*([\.][\d]+)?',sent.text).group().replace(',',''))*int(Count)),2)
                Quote_sent=get_currency(sent.text)+Quote_sent
                Received_Quote=True
            elif len(re.findall(r'[1-9](\d+)*(,\d{1,})*([\.][\d]+)?',sent.text))>=1 and get_currency(sent.text)!=0:
                Quote_sent=str(round((float(get_nearest_no(sent.text).replace(',',''))*int(Count)),2))
                Quote_sent=get_currency(sent.text)+Quote_sent
                Received_Quote=True
            else:
                print('No money component present')
                                
            if Received_Quote==True:
                if Quote_sent[-1]=='.':
                    Quote_sent=Quote_sent[0:-1]
                new_row = {'Supplier Name':supplier, 'QuoteID':Quote, 'Item Name':re.findall(QuoteRegex,sent.text)[0],'Quote Sent':Quote_sent}
                Output_df=Output_df.append(new_row,ignore_index=True)
                
        else:
            print("Too many regex strings in the statement")
Output_df

Cost of 1000 Graphite Stick Pencils – Black – b2 is 20$ per item  

Cost of 1000 Graphite Stick Pencils – Red – b2 – is 15$ per item​


Too many regex strings in the statement
​


​


 No  Next Sentence Please

Cost of 2500 Charcoal Pencils – Black – is 12$ per item














[12$]
Regards,





	


Regards,





	


 No  Next Sentence Please

Abhijith S M



	

	


Abhijith S M



	

	


 No  Next Sentence Please

abhijithsm94@outlook.com


abhijithsm94@outlook.com


 No  Next Sentence Please

Phone: + 91 - 9940243959


Phone: + 91 - 9940243959


 No  Next Sentence Please

1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India

	

 
1502 Vaikund Sundaram, Vendrasi Amman Kovil Street, Karappakam, Chennai, Tamil Nadu 60009, India

	

 
 No  Next Sentence Please

<https://twitter.com/ABHIJITH_SM>    <https://www.linkedin.com/in/abhijith-sm-723392184/>   <https://codetwocdn.azureedge.net/images/mail-signatures/generator/abstract-green/fb.png> 

,Supplier Name,QuoteID,Item Name,Quote Sent
0,Abhijith,1100,Charcoal Pencils – Black,$30000.0


In [33]:
Output_df.to_excel(writer, index=False)
Output_df.to_excel(writer, startrow=len(Output_df)+2, index=False)
writer.save()

In [94]:
doc2.sents[1].text


TypeError: 'generator' object is not subscriptable

In [106]:
for sent in doc2.sents:
    print(sent.text)
    if len(re.findall(QuoteRegex,sent.text))==0 and sent.text !='\n' :
        print(sent.text)
        print(" No  Next Sentence Please\n")
    else:
        if len(re.findall(QuoteRegex,sent.text))==1:
            Money_list=[ent for ent in nlp(sent.text).ents if ent.label_ == 'MONEY']
            print(Money_list)
            
            if len(re.findall(r'[£,$][1-9][\.\d]*(,\d+)?|[1-9][\.\d]*(,\d+)[£,$]?',sent.text))==1:
                if re.search(r'[£,$][1-9][\.\d]*(,\d+)?|[1-9][\.\d]*(,\d+)[£,$]?',sent.text).group()[-1]=='.':
                    Quote_sent=re.search(r'[£,$][1-9][\.\d]*(,\d+)?|[1-9][\.\d]*(,\d+)[£,$]?',sent.text).group()[0:-1]
                else:+)?
                    Quote_sent=re.search(r'[£,$][1-9][\.\d]*(,\d|[1-9][\.\d]*(,\d+)[£,$]?',sent.text).group()
                new_row = {'Supplier Name':supplier, 'QuoteID':Quote, 'Item Name':re.findall(QuoteRegex,sent.text)[0],'Quote Sent':Quote_sent}
                Output_df=Output_df.append(new_row,ignore_index=True)
            elif len(Money_list)>=1:
                if len(Money_list)==1:
                    for ent in nlp(sent.text).ents:
                        if len(re.findall(r'[1-9][\.\d]*(,\d+)?',ent.text))==1:
                            if re.search(r'[1-9][\.\d]*(,\d+)?',ent.text).group()[-1]=='.':
                                Quote_sent=re.search(r'[1-9][\.\d]*(,\d+)?',sent.text).group()[0:-1]
                            else:
                                Quote_sent=re.search(r'[1-9][\.\d]*(,\d+)?',sent.text).group()
                            new_row = {'Supplier Name':supplier, 'QuoteID':Quote, 'Item Name':re.findall(QuoteRegex,sent.text)[0],'Quote Sent':Quote_sent}
                            Output_df=Output_df.append(new_row,ignore_index=True)
                        else:
                            print('Unable to find actual money in money entity')
                else:
                    print("To money eqts in sentence")
            else:
                print('Unable to find money in sentence')
                

            
                    #print(re.findall(QuoteRegex,sent.text)[0]+'     '+ re.search(r'[£,$][1-9][\.\d]*(,\d+)?',sent.text).group())
        else:
            print('To many item names in a sentence')
Output_df

SyntaxError: invalid syntax (<ipython-input-106-b787ae587b59>, line 14)

In [105]:
def get_currency(text):
    Flag=0
    df_Symbols = pd.read_excel('C:\\Users\\slice\\NLP POC\\NLP_Quotation\\Input\\Currency Symbols.xlsx')
    for index, row in df_Symbols.iterrows():
        if str(row['Text']).lower() in text.lower():
            Flag=1
            return str(row['Symbol']);
            break
    if Flag==0:
        return 0;


In [55]:
get_currency('doasd')

0

In [104]:
def is_per(text):
    Flag=0
    df_strings= pd.read_excel('C:\\Users\\slice\\NLP POC\\NLP_Quotation\\Input\\per.xlsx')
    for index, row in df_strings.iterrows():
        if str(row['String']).lower() in text.lower():
            Flag=1
            return True;
    if Flag==0:
        return False;
    

In [58]:
is_per(' as per each')

True

In [65]:
round(float('3,000.05')*3,2)

ValueError: could not convert string to float: '3,000.05'